In [ ]:
#@title 🎧 Download Narration Audio & Play Introduction
import os as _os
if not _os.path.exists("/content/narration"):
    !pip install -q gdown
    import gdown
    gdown.download(id="1VLFlUCL9vCbtUm2KD6CbL9RdcA0VWmBm", output="/content/narration.zip", quiet=False)
    !unzip -q /content/narration.zip -d /content/narration
    !rm /content/narration.zip
    print(f"Loaded {len(_os.listdir('/content/narration'))} narration segments")
else:
    print("Narration audio already loaded.")

from IPython.display import Audio, display
display(Audio("/content/narration/00_intro.mp3"))

In [ ]:
# 🔧 Setup: Run this cell first!
# Check GPU availability and install dependencies

import torch
import sys

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️ No GPU detected. Some cells may run slowly.")
    print("   Go to Runtime → Change runtime type → GPU")

print(f"\n📦 Python {sys.version.split()[0]}")
print(f"🔥 PyTorch {torch.__version__}")

# Set random seeds for reproducibility
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"🎲 Random seed set to {SEED}")

%matplotlib inline

In [ ]:
#@title 🎧 Listen: Why It Matters
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_why_it_matters.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

# Diffusion Action Decoder: Sculpting Trajectories from Noise

*Part 4 of the Vizuara series on VLAs for Autonomous Driving*
*Estimated time: 75 minutes | Runs on: Google Colab (T4 GPU)*

# 🤖 AI Teaching Assistant

Need help with this notebook? Open the **AI Teaching Assistant** — it has already read this entire notebook and can help with concepts, code, and exercises.

**[👉 Open AI Teaching Assistant](https://course-creator-brown.vercel.app/courses/vla-autonomous-driving/practice/4/assistant)**

*Tip: Open it in a separate tab and work through this notebook side-by-side.*


## 1. Why Does This Matter?

In Notebook 2, we built an action tokenizer that discretizes continuous driving commands into tokens. It works — but a steering angle of 14.8 degrees and 15.2 degrees both map to the same token. At highway speeds, that 0.4-degree difference means drifting into an adjacent lane over a few seconds. For safety-critical driving, discretization artifacts are not acceptable.

NVIDIA's Alpamayo takes a radically different approach: a **diffusion action decoder** (2.3B params) that generates infinitely smooth, continuous trajectories without any discretization. It takes internal representations from the VLM backbone and produces 64 waypoints at 10 Hz — 6.4 seconds of future driving, each with position (x, y, z) and rotation.

The core idea: start with random noise and iteratively refine it into a clean trajectory — like sculpting a statue out of marble. By the end, you will build a conditional trajectory diffusion model, train it, and watch an animation of noise being sculpted into smooth driving paths.

In [ ]:
#@title 🎧 Listen: Intuition
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_intuition.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## 2. Building Intuition

**Analogy: The sculptor.** You cannot carve a perfect statue in a single strike. Instead, you work iteratively — remove large chunks for the rough form, refine contours, then smooth the surface. Diffusion models work the same way: start with pure noise (the marble) and iteratively denoise until a clean trajectory emerges.

**Why not predict the trajectory directly?** Consider a car at a T-intersection. It could turn left, right, or go straight. A regression model trained with MSE loss would average these options and output a trajectory that goes straight into the building across the street. A diffusion model captures the *full distribution* — each sample can produce any valid option. This **multimodality** is why diffusion is the method of choice for trajectory generation.

**The key insight.** Instead of learning to generate trajectories from scratch (very hard), we learn to *remove noise* (much easier). If I show you a slightly noisy trajectory and ask "what noise was added?", that is simpler than "generate a perfect trajectory from nothing." Chain 100 such denoising steps together and we go from pure noise to a clean trajectory.

### Think About This

Why 64 waypoints at 10 Hz? That is 6.4 seconds. At highway speeds (30 m/s), the model plans ~192 meters ahead — enough for lane changes and merges, but not so far that predictions become unreliable.

In [ ]:
#@title 🎧 Listen: Math
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_math.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## 3. The Mathematics

### 3.1 Forward Diffusion (Adding Noise)

$$q(x_t | x_0) = \mathcal{N}(x_t; \sqrt{\bar{\alpha}_t} \, x_0, (1 - \bar{\alpha}_t) \mathbf{I})$$

We can jump from clean trajectory $x_0$ to any noise level $t$ in one step. $\bar{\alpha}_t$ controls signal retention: at $t=0$, $\bar{\alpha}_0 \approx 1$ (unchanged); at $t=T$, $\bar{\alpha}_T \approx 0$ (pure noise). In practice:

$$x_t = \sqrt{\bar{\alpha}_t} \cdot x_0 + \sqrt{1 - \bar{\alpha}_t} \cdot \epsilon, \quad \epsilon \sim \mathcal{N}(0, \mathbf{I})$$

**Worked example.** Waypoint $x_0 = [2.0, 0.5]$, $\sqrt{\bar{\alpha}_t} = 0.8$, $\sqrt{1-\bar{\alpha}_t} = 0.6$, noise $\epsilon = [0.3, -0.1]$:

$$x_t = 0.8 \times [2.0, 0.5] + 0.6 \times [0.3, -0.1] = [1.6, 0.4] + [0.18, -0.06] = [1.78, 0.34]$$

### 3.2 Noise Schedule

$$\beta_t = \beta_{\min} + \frac{t}{T}(\beta_{\max} - \beta_{\min}), \quad \alpha_t = 1 - \beta_t, \quad \bar{\alpha}_t = \prod_{s=1}^{t} \alpha_s$$

$\beta_t$ grows linearly from $0.0001$ to $0.02$. The cumulative product $\bar{\alpha}_t$ gives total signal remaining.

**Worked example.** $T=100$: at $t=0$, $\bar{\alpha}_0 = 0.9999$ (99.99% signal). At $t=50$, $\bar{\alpha}_{50} \approx 0.62$ (62%). At $t=99$, $\bar{\alpha}_{99} \approx 0.006$ (pure noise).

### 3.3 Training Objective

$$\mathcal{L} = \mathbb{E}_{t, x_0, \epsilon} \left[ \| \epsilon - \epsilon_\theta(x_t, t, c) \|^2 \right]$$

Given noisy trajectory $x_t$, timestep $t$, and conditioning $c$ (scene features), predict the noise $\epsilon$. Just MSE between predicted and actual noise.

**Worked example.** True $\epsilon = [0.3, -0.1]$, predicted $\hat{\epsilon} = [0.25, -0.08]$. Loss: $(0.3-0.25)^2 + (-0.1+0.08)^2 = 0.0029$.

### 3.4 Sampling (Denoising)

$$x_{t-1} = \frac{1}{\sqrt{\alpha_t}} \left( x_t - \frac{\beta_t}{\sqrt{1 - \bar{\alpha}_t}} \epsilon_\theta(x_t, t, c) \right) + \sigma_t z$$

Starting from $x_T$ (noise), apply this repeatedly. The predicted noise tells us which direction to step; $\sigma_t z$ adds stochasticity for diversity.

**Worked example.** At $t=50$: $x_{50}=[0.5,1.2]$, $\hat{\epsilon}=[0.3,-0.4]$, $\beta_{50}=0.0101$, $\sqrt{\alpha_{50}}=0.9949$, $\sqrt{1-\bar{\alpha}_{50}}=0.616$:

$$x_{49} = \frac{1}{0.9949}([0.5-0.0049, 1.2+0.0066]) + \sigma z = [0.4975, 1.2127] + \text{small noise}$$

One step closer. After 100 steps: a smooth driving path.

In [ ]:
#@title 🎧 Listen: Setup And Schedule
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_setup_and_schedule.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## 4. Let Us Build It

### Setup

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
%matplotlib inline

torch.manual_seed(42)
np.random.seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

### 4.1 Noise Schedule

In [ ]:
class NoiseSchedule:
    """Linear noise schedule — precomputes all quantities for training and sampling."""
    def __init__(self, n_timesteps=100, beta_min=1e-4, beta_max=0.02):
        self.n_timesteps = n_timesteps
        self.betas = torch.linspace(beta_min, beta_max, n_timesteps)
        self.alphas = 1.0 - self.betas
        self.alpha_bars = torch.cumprod(self.alphas, dim=0)
        self.sqrt_alpha_bars = torch.sqrt(self.alpha_bars)
        self.sqrt_one_minus_alpha_bars = torch.sqrt(1.0 - self.alpha_bars)
        self.sqrt_alphas = torch.sqrt(self.alphas)

In [ ]:
# 📊 Visualize the noise schedule
schedule = NoiseSchedule(n_timesteps=100)
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].plot(schedule.betas.numpy(), 'b-', linewidth=2)
axes[0].set_xlabel('Timestep t'); axes[0].set_ylabel(r'$\beta_t$')
axes[0].set_title(r'Noise Rate $\beta_t$'); axes[0].grid(True, alpha=0.3)

axes[1].plot(schedule.alpha_bars.numpy(), 'r-', linewidth=2)
axes[1].set_xlabel('Timestep t'); axes[1].set_ylabel(r'$\bar{\alpha}_t$')
axes[1].set_title(r'Cumulative Signal $\bar{\alpha}_t$'); axes[1].grid(True, alpha=0.3)

axes[2].plot(schedule.sqrt_one_minus_alpha_bars.numpy(), 'g-', linewidth=2)
axes[2].set_xlabel('Timestep t'); axes[2].set_ylabel(r'$\sqrt{1-\bar{\alpha}_t}$')
axes[2].set_title('Noise Level'); axes[2].grid(True, alpha=0.3)

plt.tight_layout(); plt.show()
print("At t=0, signal preserved. At t=99, pure noise. This is our sculpting plan.")

In [ ]:
#@title 🎧 Listen: Forward Diffusion
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_forward_diffusion.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

### 4.2 Forward Diffusion

In [ ]:
def forward_diffusion(x_0, t, schedule, noise=None):
    """Add noise to clean trajectories: x_t = sqrt(a_bar)*x_0 + sqrt(1-a_bar)*noise."""
    if noise is None:
        noise = torch.randn_like(x_0)
    sqrt_alpha_bar = schedule.sqrt_alpha_bars[t].reshape(-1, 1, 1)
    sqrt_one_minus = schedule.sqrt_one_minus_alpha_bars[t].reshape(-1, 1, 1)
    x_t = sqrt_alpha_bar * x_0 + sqrt_one_minus * noise
    return x_t, noise

In [ ]:
# 📊 Watch a trajectory dissolve into noise
t_param = np.linspace(0, 1, 64)
clean_traj = np.stack([t_param * 50, 5 * np.sin(2 * np.pi * t_param)], axis=1)
x_0 = torch.FloatTensor(clean_traj).unsqueeze(0)

timesteps_to_show = [0, 10, 25, 50, 75, 99]
fig, axes = plt.subplots(1, 6, figsize=(20, 3))
torch.manual_seed(42)
for ax, ts in zip(axes, timesteps_to_show):
    x_t, _ = forward_diffusion(x_0, torch.tensor([ts]), schedule)
    traj = x_t.squeeze(0).numpy()
    ax.plot(clean_traj[:, 0], clean_traj[:, 1], 'g-', alpha=0.3)
    ax.plot(traj[:, 0], traj[:, 1], 'r-', linewidth=1.5)
    ax.set_title(f't = {ts}', fontweight='bold')
    ax.set_xlim(-20, 70); ax.set_ylim(-20, 20)
    ax.grid(True, alpha=0.3); ax.set_aspect('equal')
plt.suptitle("Forward Diffusion: Clean → Noise", fontsize=14, fontweight='bold')
plt.tight_layout(); plt.show()

In [ ]:
#@title 🎧 Listen: Denoiser Network
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/06_denoiser_network.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

### 4.3 Denoising Network

In [ ]:
class SinusoidalPosEmb(nn.Module):
    """Sinusoidal timestep embedding (same trick as original Transformer)."""
    def __init__(self, dim):
        super().__init__()
        self.dim = dim
    def forward(self, t):
        half = self.dim // 2
        emb = torch.exp(torch.arange(half, device=t.device) * -(np.log(10000) / (half - 1)))
        emb = t.unsqueeze(1) * emb.unsqueeze(0)
        return torch.cat([emb.sin(), emb.cos()], dim=-1)

class TrajectoryDenoiser(nn.Module):
    """Predict noise in a trajectory, conditioned on timestep and scene features."""
    def __init__(self, traj_dim=2, hidden_dim=128, n_waypoints=64, condition_dim=32):
        super().__init__()
        self.n_waypoints = n_waypoints
        self.time_embed = nn.Sequential(
            SinusoidalPosEmb(hidden_dim), nn.Linear(hidden_dim, hidden_dim), nn.GELU())
        self.cond_proj = nn.Linear(condition_dim, hidden_dim)
        input_dim = n_waypoints * traj_dim
        self.net = nn.Sequential(
            nn.Linear(input_dim + hidden_dim * 2, hidden_dim * 4), nn.GELU(),
            nn.Linear(hidden_dim * 4, hidden_dim * 4), nn.GELU(),
            nn.Linear(hidden_dim * 4, hidden_dim * 2), nn.GELU(),
            nn.Linear(hidden_dim * 2, input_dim))

    def forward(self, x_t, t, condition):
        batch = x_t.size(0)
        x_flat = x_t.reshape(batch, -1)
        t_emb = self.time_embed(t.float())
        c_emb = self.cond_proj(condition)
        noise_pred = self.net(torch.cat([x_flat, t_emb, c_emb], dim=-1))
        return noise_pred.reshape(batch, self.n_waypoints, 2)

In [ ]:
# Sanity check
model_test = TrajectoryDenoiser().to(device)
out = model_test(torch.randn(4,64,2).to(device), torch.tensor([10,20,50,90]).to(device),
                 torch.randn(4,32).to(device))
print(f"Input: (4, 64, 2) → Output: {out.shape}")
print(f"Parameters: {sum(p.numel() for p in model_test.parameters()):,}")

In [ ]:
#@title 🎧 Listen: Synthetic Data
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/07_synthetic_data.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

### 4.4 Synthetic Trajectory Data

We generate five maneuver types — straight, lane change, left/right turns, S-curves — with random speed and amplitude variation. Each trajectory gets a condition vector (one-hot type + parameters), mimicking how Alpamayo conditions on VLM scene features.

In [ ]:
def generate_trajectory_dataset(n_samples=5000, n_waypoints=64):
    """Generate diverse 2D driving trajectories with condition labels."""
    trajectories, conditions = [], []
    types = ['straight', 'lane_change', 'left_turn', 'right_turn', 's_curve']
    for _ in range(n_samples):
        t = np.linspace(0, 1, n_waypoints)
        idx = np.random.randint(len(types))
        speed = np.random.uniform(0.8, 1.2)
        amp = np.random.uniform(0.8, 1.2)
        if types[idx] == 'straight':
            x, y = t * 50 * speed, np.random.normal(0, 0.3, n_waypoints).cumsum() * 0.1
        elif types[idx] == 'lane_change':
            x, y = t * 50 * speed, 3.5 * amp / (1 + np.exp(-10 * (t - 0.5)))
        elif types[idx] == 'left_turn':
            r, theta = 20*amp, t*np.pi/2; x, y = r*np.sin(theta), r*(1-np.cos(theta))
        elif types[idx] == 'right_turn':
            r, theta = 20*amp, t*np.pi/2; x, y = r*np.sin(theta), -r*(1-np.cos(theta))
        else:
            x, y = t * 50 * speed, 4.0 * amp * np.sin(2 * np.pi * t)
        x += np.random.normal(0, 0.05, n_waypoints)
        y += np.random.normal(0, 0.05, n_waypoints)
        cond = np.zeros(32); cond[idx] = 1.0; cond[5] = speed; cond[6] = amp
        trajectories.append(np.stack([x, y], axis=1)); conditions.append(cond)
    return torch.FloatTensor(np.array(trajectories)), torch.FloatTensor(np.array(conditions))

torch.manual_seed(42); np.random.seed(42)
trajectories, conditions = generate_trajectory_dataset(5000, 64)
print(f"Dataset: {trajectories.shape[0]} trajectories, {trajectories.shape[1]} waypoints each")

In [ ]:
# 📊 Visualize training data
fig, axes = plt.subplots(1, 5, figsize=(20, 4))
names = ['Straight', 'Lane Change', 'Left Turn', 'Right Turn', 'S-Curve']
for col in range(5):
    for j, traj in enumerate(trajectories[conditions[:, col] == 1.0][:8]):
        axes[col].plot(traj[:, 0].numpy(), traj[:, 1].numpy(), alpha=0.5, linewidth=1)
    axes[col].set_title(names[col], fontweight='bold'); axes[col].grid(True, alpha=0.3)
    axes[col].set_aspect('equal')
plt.suptitle("Training Data: Five Maneuver Types", fontsize=14, fontweight='bold')
plt.tight_layout(); plt.show()

We need to normalize the trajectories before training — diffusion models work best when data has zero mean and unit variance.

In [ ]:
# Normalize (diffusion works best with zero mean, unit variance)
traj_mean = trajectories.mean(dim=(0, 1), keepdim=True)
traj_std = trajectories.std(dim=(0, 1), keepdim=True) + 1e-6
trajectories_norm = (trajectories - traj_mean) / traj_std
print(f"Normalized — mean: {trajectories_norm.mean():.4f}, std: {trajectories_norm.std():.4f}")

In [ ]:
#@title 🎧 Listen: Training
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/08_training.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

### 4.5 Training

For each batch: sample random timesteps, add noise, predict noise, compute MSE, backpropagate. That is the entire algorithm.

In [ ]:
denoiser = TrajectoryDenoiser(traj_dim=2, hidden_dim=128, n_waypoints=64, condition_dim=32).to(device)
schedule = NoiseSchedule(n_timesteps=100)
optimizer = torch.optim.Adam(denoiser.parameters(), lr=1e-3)
batch_size, n_epochs = 128, 50
losses = []
print(f"Parameters: {sum(p.numel() for p in denoiser.parameters()):,}")

for epoch in range(n_epochs):
    denoiser.train()
    perm = torch.randperm(len(trajectories_norm))
    epoch_loss, n_batches = 0, 0
    for i in range(0, len(trajectories_norm), batch_size):
        idx = perm[i:i+batch_size]
        x_0 = trajectories_norm[idx].to(device)
        cond = conditions[idx].to(device)
        t = torch.randint(0, schedule.n_timesteps, (len(idx),), device=device)
        noise = torch.randn_like(x_0)
        sqrt_ab = schedule.sqrt_alpha_bars[t].reshape(-1,1,1).to(device)
        sqrt_1mab = schedule.sqrt_one_minus_alpha_bars[t].reshape(-1,1,1).to(device)
        x_t = sqrt_ab * x_0 + sqrt_1mab * noise
        loss = F.mse_loss(denoiser(x_t, t, cond), noise)
        optimizer.zero_grad(); loss.backward(); optimizer.step()
        epoch_loss += loss.item(); n_batches += 1
    losses.append(epoch_loss / n_batches)
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1:3d}/{n_epochs} | Loss: {losses[-1]:.6f}")

In [ ]:
# 📊 Training curve
plt.figure(figsize=(8, 4))
plt.plot(losses, 'b-', linewidth=2)
plt.xlabel('Epoch'); plt.ylabel('MSE Loss')
plt.title('Diffusion Denoiser Training'); plt.grid(True, alpha=0.3)
plt.tight_layout(); plt.show()
print("Lower loss = better noise prediction = smoother trajectories.")

In [ ]:
#@title 🎧 Listen: Sampling
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/09_sampling.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

### 4.6 Sampling (Reverse Diffusion)

Start from pure noise. At each step, predict noise and subtract it. After 100 steps: a clean trajectory.

In [ ]:
@torch.no_grad()
def sample_trajectories(denoiser, schedule, condition, n_samples=1,
                        n_waypoints=64, device='cpu', return_intermediates=False):
    """Generate trajectories by iterative denoising."""
    denoiser.eval()
    x = torch.randn(n_samples, n_waypoints, 2, device=device)
    intermediates = [x.cpu().clone()]
    for t in reversed(range(schedule.n_timesteps)):
        t_batch = torch.full((n_samples,), t, device=device, dtype=torch.long)
        noise_pred = denoiser(x, t_batch, condition)
        beta_t = schedule.betas[t].to(device)
        sqrt_alpha_t = schedule.sqrt_alphas[t].to(device)
        sqrt_1m_ab = schedule.sqrt_one_minus_alpha_bars[t].to(device)
        x = (1 / sqrt_alpha_t) * (x - (beta_t / sqrt_1m_ab) * noise_pred)
        if t > 0:
            x = x + torch.sqrt(beta_t) * torch.randn_like(x)
        if return_intermediates and t % 5 == 0:
            intermediates.append(x.cpu().clone())
    return (x, intermediates) if return_intermediates else x

In [ ]:
#@title 🎧 Listen: Todo1 Forward
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/10_todo1_forward.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## 5. Your Turn

### TODO 1: Implement Forward Diffusion

Implement $x_t = \sqrt{\bar{\alpha}_t} \cdot x_0 + \sqrt{1 - \bar{\alpha}_t} \cdot \epsilon$.

In [ ]:
def my_forward_diffusion(x_0, t, sqrt_alpha_bars, sqrt_one_minus_alpha_bars):
    """
    Args:
        x_0: (batch, n_waypoints, 2) — clean trajectories
        t: (batch,) — timestep indices
        sqrt_alpha_bars, sqrt_one_minus_alpha_bars: precomputed schedule values
    Returns: (x_t, noise) — noisy trajectories and the noise added

    Hints: torch.randn_like for noise, index with t, reshape to (-1,1,1) for broadcasting
    """
    # ============ YOUR CODE HERE ============
    noise = ???       # Step 1: Sample noise ~ N(0, I)
    sqrt_ab = ???     # Step 2: Look up sqrt(alpha_bar_t), reshape to (batch, 1, 1)
    sqrt_1mab = ???   # Step 3: Look up sqrt(1 - alpha_bar_t), reshape
    x_t = ???         # Step 4: x_t = sqrt_ab * x_0 + sqrt_1mab * noise
    # ========================================
    return x_t, noise

In [ ]:
# ✅ Verification
torch.manual_seed(123)
x_test = torch.randn(4, 64, 2)
t_test = torch.tensor([0, 25, 50, 99])
x_t, eps = my_forward_diffusion(x_test, t_test, schedule.sqrt_alpha_bars,
                                 schedule.sqrt_one_minus_alpha_bars)
assert x_t.shape == x_test.shape, f"Wrong shape: {x_t.shape}"
diff_t0 = (x_t[0] - x_test[0]).abs().max().item()
assert diff_t0 < 0.15, f"At t=0, x_t should ≈ x_0 (diff={diff_t0:.3f})"
assert (x_t[3] - x_test[3]).abs().mean().item() > 0.5, "At t=99, should be mostly noise"
print(f"✅ Forward diffusion correct! t=0 deviation: {diff_t0:.4f}")

In [ ]:
#@title 🎧 Listen: Todo2 Denoise Step
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/11_todo2_denoise_step.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

### TODO 2: Implement a Denoising Step

Implement one reverse step: $x_{t-1} = \frac{1}{\sqrt{\alpha_t}}(x_t - \frac{\beta_t}{\sqrt{1-\bar{\alpha}_t}}\epsilon_\theta) + \sigma_t z$

In [ ]:
def my_denoise_step(x_t, t, noise_pred, schedule):
    """
    One reverse diffusion step: x_t → x_{t-1}.
    Args:
        x_t: (batch, n_waypoints, 2), t: int, noise_pred: (batch, n_waypoints, 2)
    Hints: Get beta_t, sqrt_alpha_t, sqrt_1m_alpha_bar from schedule.
           If t > 0, add sigma_t * z where sigma_t = sqrt(beta_t).
    """
    # ============ YOUR CODE HERE ============
    beta_t = ???
    sqrt_alpha_t = ???
    sqrt_1m_alpha_bar = ???
    x_prev = ???  # mean = (1/sqrt_alpha_t) * (x_t - (beta_t/sqrt_1m_alpha_bar) * noise_pred)
    if t > 0:
        x_prev = ???  # add sigma_t * z
    # ========================================
    return x_prev

In [ ]:
# ✅ Verification
torch.manual_seed(456)
x_noisy = torch.randn(2, 64, 2)
fake_noise = torch.randn(2, 64, 2)
x_out = my_denoise_step(x_noisy, t=50, noise_pred=fake_noise, schedule=schedule)
assert x_out.shape == x_noisy.shape, f"Wrong shape: {x_out.shape}"
assert not torch.allclose(x_out, x_noisy, atol=1e-6), "Output should differ from input"
x_final = my_denoise_step(x_noisy, t=0, noise_pred=fake_noise, schedule=schedule)
assert x_final.shape == x_noisy.shape
print("✅ Denoising step correct! You have all pieces for a full sampling loop.")

In [ ]:
#@title 🎧 Listen: Todo3 Sampling Loop
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/12_todo3_sampling_loop.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

### TODO 3 (Bonus): Full Sampling Loop

In [ ]:
def my_sample(denoiser, schedule, condition, n_waypoints=64, device='cpu'):
    """Generate a trajectory from pure noise using your denoise step."""
    denoiser.eval()
    # ============ YOUR CODE HERE ============
    x = ???  # Start from pure noise: (1, n_waypoints, 2)
    with torch.no_grad():
        for t in reversed(range(schedule.n_timesteps)):
            t_batch = ???       # (1,) tensor of current timestep
            noise_pred = ???    # denoiser(x, t_batch, condition)
            x = ???             # my_denoise_step(x, t, noise_pred, schedule)
    # ========================================
    return x

In [ ]:
# Test your sampling loop
cond_test = torch.zeros(1, 32).to(device)
cond_test[0, 0] = 1.0; cond_test[0, 5] = 1.0; cond_test[0, 6] = 1.0
result = my_sample(denoiser, schedule, cond_test, device=device)
result_np = (result.cpu() * traj_std + traj_mean).squeeze(0).numpy()

plt.figure(figsize=(8, 3))
plt.plot(result_np[:, 0], result_np[:, 1], 'b-o', markersize=2, linewidth=1.5)
plt.plot(result_np[0, 0], result_np[0, 1], 'go', markersize=10, label='Start')
plt.plot(result_np[-1, 0], result_np[-1, 1], 'ro', markersize=10, label='End')
plt.title("Your Generated Trajectory", fontweight='bold')
plt.legend(); plt.grid(True, alpha=0.3); plt.axis('equal'); plt.tight_layout(); plt.show()
print("Smooth trajectory? Your diffusion model works end to end!")

In [ ]:
#@title 🎧 Listen: All Maneuvers And Animation
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/13_all_maneuvers_and_animation.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## 6. Final Output: All Maneuver Types + Animation

In [ ]:
# 📊 Generate all five maneuver types
type_labels = ['Straight', 'Lane Change', 'Left Turn', 'Right Turn', 'S-Curve']
fig, axes = plt.subplots(2, 5, figsize=(20, 8))

for col, label in enumerate(type_labels):
    cond = torch.zeros(1, 32).to(device)
    cond[0, col] = 1.0; cond[0, 5] = 1.0; cond[0, 6] = 1.0
    final_traj, intermediates = sample_trajectories(
        denoiser, schedule, cond, n_samples=1, n_waypoints=64,
        device=device, return_intermediates=True)
    final_np = (final_traj.cpu() * traj_std + traj_mean).squeeze(0).numpy()

    axes[0, col].plot(final_np[:, 0], final_np[:, 1], 'b-', linewidth=2)
    axes[0, col].plot(final_np[0, 0], final_np[0, 1], 'go', markersize=8)
    axes[0, col].plot(final_np[-1, 0], final_np[-1, 1], 'ro', markersize=8)
    axes[0, col].set_title(label, fontweight='bold')
    axes[0, col].grid(True, alpha=0.3); axes[0, col].set_aspect('equal')

    n_show = min(6, len(intermediates))
    idxs = np.linspace(0, len(intermediates)-1, n_show, dtype=int)
    colors = plt.cm.Blues(np.linspace(0.2, 1.0, n_show))
    for j, si in enumerate(idxs):
        ti = (intermediates[si] * traj_std + traj_mean).squeeze(0).numpy()
        axes[1, col].plot(ti[:, 0], ti[:, 1], '-', color=colors[j],
                         alpha=0.4+0.6*(j/(n_show-1)), linewidth=1)
    axes[1, col].set_title('Denoising Steps'); axes[1, col].grid(True, alpha=0.3)
    axes[1, col].set_aspect('equal')

plt.suptitle("Diffusion: From Noise to Smooth Driving Paths", fontsize=14, fontweight='bold')
plt.tight_layout(); plt.show()
print("This is exactly what we want — diverse, smooth trajectories for each maneuver type.")

Now the showstopper: an animation of the full denoising process.

In [ ]:
# Capture all 100 denoising steps for a lane change
cond_anim = torch.zeros(1, 32).to(device)
cond_anim[0, 1] = 1.0; cond_anim[0, 5] = 1.0; cond_anim[0, 6] = 1.0
denoiser.eval(); torch.manual_seed(42)
x = torch.randn(1, 64, 2, device=device)
all_steps = [x.cpu().clone()]
with torch.no_grad():
    for t in reversed(range(schedule.n_timesteps)):
        t_batch = torch.full((1,), t, device=device, dtype=torch.long)
        noise_pred = denoiser(x, t_batch, cond_anim)
        beta_t = schedule.betas[t].to(device)
        x = (1/schedule.sqrt_alphas[t].to(device)) * (
            x - (beta_t/schedule.sqrt_one_minus_alpha_bars[t].to(device)) * noise_pred)
        if t > 0: x = x + torch.sqrt(beta_t) * torch.randn_like(x)
        all_steps.append(x.cpu().clone())
print(f"Captured {len(all_steps)} snapshots")

Now let us animate the full denoising process — watch a lane-change trajectory materialize from pure noise, one step at a time.

In [ ]:
# 🎬 Animate: noise → trajectory
fig_anim, ax_anim = plt.subplots(figsize=(10, 5))
def update(frame):
    ax_anim.clear()
    traj = (all_steps[frame] * traj_std + traj_mean).squeeze(0).numpy()
    ax_anim.plot(traj[:, 0], traj[:, 1], 'b-o', markersize=2, linewidth=1.5)
    ax_anim.plot(traj[0, 0], traj[0, 1], 'go', markersize=8)
    ax_anim.plot(traj[-1, 0], traj[-1, 1], 'ro', markersize=8)
    p = frame / (len(all_steps) - 1)
    phase = ("Pure Noise" if p < 0.05 else "Rough Shape..." if p < 0.3 else
             "Sculpting..." if p < 0.7 else "Refining..." if p < 0.95 else "Clean Trajectory!")
    ax_anim.set_title(f"Step {frame}/{len(all_steps)-1} — {phase}", fontsize=13, fontweight='bold')
    ax_anim.set_xlim(-30, 80); ax_anim.set_ylim(-15, 15)
    ax_anim.grid(True, alpha=0.3); ax_anim.set_aspect('equal')
    ax_anim.set_xlabel('x (meters)'); ax_anim.set_ylabel('y (meters)')
anim = animation.FuncAnimation(fig_anim, update, frames=len(all_steps), interval=50)
HTML(anim.to_jshtml())

In [ ]:
#@title 🎧 Listen: Multimodality
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/14_multimodality.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## 7. Multimodality: The Killer Feature

In [ ]:
# 📊 10 trajectories from the SAME condition — demonstrating distribution capture
cond_multi = torch.zeros(10, 32).to(device)
cond_multi[:, 2] = 1.0; cond_multi[:, 5] = 1.0; cond_multi[:, 6] = 1.0
torch.manual_seed(0)
multi = sample_trajectories(denoiser, schedule, cond_multi, n_samples=10,
                            n_waypoints=64, device=device)
multi = (multi.cpu() * traj_std + traj_mean).numpy()

plt.figure(figsize=(8, 8))
for i in range(10):
    c = plt.cm.tab10(i/10)
    plt.plot(multi[i,:,0], multi[i,:,1], '-', color=c, linewidth=1.5, alpha=0.7, label=f'Sample {i+1}')
    plt.plot(multi[i,0,0], multi[i,0,1], 'o', color=c, markersize=6)
plt.title("10 Left Turns from Same Condition\n(Different noise → different valid trajectories)",
          fontsize=13, fontweight='bold')
plt.xlabel('x (m)'); plt.ylabel('y (m)')
plt.legend(fontsize=8); plt.grid(True, alpha=0.3); plt.axis('equal')
plt.tight_layout(); plt.show()
print("All valid left turns, all different. Regression would give ONE average (wrong) trajectory.")
print("Diffusion captures the full distribution — this is exactly what we want.")

In [ ]:
#@title 🎧 Listen: Closing
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/15_closing.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## 8. Reflection and Next Steps

### What We Built

A complete diffusion action decoder: noise schedule, forward diffusion, denoising network, and reverse sampling. The same principle powers Alpamayo's 2.3B-parameter decoder — differences are in scale (MLP vs. transformer), dimensionality (2D vs. 3D poses), and conditioning (one-hot vs. VLM features). The core algorithm is identical.

### Reflection Questions

1. We used 100 steps. With 10? Faster but lower quality. DDIM achieves good quality in 10 steps — critical for real-time driving at 10 Hz.
2. Why 6.4 seconds? At 30 m/s, that is ~192m ahead. How would you choose the horizon for city vs. highway driving?
3. We showed multimodality, but a real car must pick ONE trajectory. How would you score candidates for safety, comfort, and efficiency?

### Optional Challenges

1. **DDIM Sampling:** Deterministic sampler — reduce from 100 to 10 steps with minimal quality loss.
2. **Classifier-Free Guidance:** Drop condition 10% during training. At inference: $\hat{\epsilon} = \epsilon_{\text{unc}} + s(\epsilon_{\text{cond}} - \epsilon_{\text{unc}})$ for stronger conditioning.
3. **3D Trajectories:** Extend to (x, y, z) for hills and overpasses — what Alpamayo actually outputs.
4. **Cosine Schedule:** $\bar{\alpha}_t = \cos(\frac{t/T+s}{1+s}\frac{\pi}{2})^2$ — spends more steps at intermediate noise levels.